In [40]:
import pandas as pd
import altair as alt
from altair_saver import save

In [55]:
df = pd.read_excel("Cost of car insurance by state.xlsx", header=1, thousands = "r", usecols=["Rank", "State", "Full coverage", "Minimum coverage", "Difference"])
df.head()

,Rank,State,Full coverage,Minimum coverage,Difference
0,1,Michigan,8723,5282,3441
1,2,Rhode Island,3847,1589,2258
2,3,Louisiana,3525,1329,2196
3,4,Kentucky,3418,1338,2079
4,5,Florida,3370,2565,805


In [23]:
# Adds a $ symbol and thousands separators
def convert(column):
    return "$"+ "{:,}".format(column)

In [24]:
# Adds additional columns with formatted values which will be used as labels
df[["Full cov $", "Min cov $"]] = df[["Full coverage", "Minimum coverage"]].applymap(convert)
df.head()

,Rank,State,Full coverage,Minimum coverage,Difference,Full cov $,Min cov $
0,1,Michigan,8723,5282,3441,"$8,723","$5,282"
1,2,Rhode Island,3847,1589,2258,"$3,847","$1,589"
2,3,Louisiana,3525,1329,2196,"$3,525","$1,329"
3,4,Kentucky,3418,1338,2079,"$3,418","$1,338"
4,5,Florida,3370,2565,805,"$3,370","$2,565"


In [32]:
# Highlight action
selection = alt.selection_single(fields=["State"])

# Highlight condition
color1 = alt.condition(selection, alt.value("#e86f6f"), alt.value("lightgray"))

# Full coverage bar chart
full_coverage = (
    alt.Chart(df)
    .mark_bar(size = 14) # Changes the size of the individual bars
    .encode(
        x=alt.X("Full coverage", axis=None), # removes the x-axis
        y=alt.Y("State", sort="-x", title=""),
        tooltip=["State", "Full coverage"],
        color=color1, # add the highlight condition
    )
)

# Full coverage labels
full_coverage_text = full_coverage.mark_text(
    align="left",
    baseline="middle",
    dx=3,  # Nudges text to right so it doesn't appear on top of the bar
).encode(text="Full cov $:N")

# Minimum coverage bar chart
min_coverage = (
    alt.Chart(df)
    .mark_bar(size = 14)
    .encode(
        x=alt.X("Minimum coverage", axis=None),
        y=alt.Y("State", sort="-x", title=""), # Removes the "State" field label
        tooltip=["State", "Minimum coverage"],
        color=color1,
    )
)

# Minimum coverage labels
min_coverage_text = min_coverage.mark_text(
    align="left",
    baseline="middle",
    dx=3, 
).encode(text="Min cov $:N")

# Displays the two charts next to each other
chart = alt.hconcat(
    (full_coverage + full_coverage_text) # Adds bars and labels
    .add_selection(selection) # Binds the highlighting action
    .properties(title="Full coverage"), # Adds a title
    (min_coverage + min_coverage_text)
    .add_selection(selection)
    .properties(title="Minimum coverage"),
).configure_axis(grid=False).configure_view(strokeOpacity=0).resolve_scale( # Removes grid lines and uses the same x axis scale for both charts
    x="shared"
).properties(
    title="Cost of car insurance by state"
).configure_title(
    anchor="start", fontSize=14
).configure_axisY( # left aligns y-axis labels
    titleAngle=0, 
    titleY=-10,
    titleX=-60,
    labelPadding=160, 
    labelAlign='left'
)

chart

alt.HConcatChart(...)

In [54]:
# This renders the chart on GitHub

alt.renderers.enable('altair_saver', fmts=['vega-lite', 'html'])
chart.display()

alt.HConcatChart(...)

In [53]:
chart.save("car_insurance.html")